In [ ]:
import sys
import os

# Insert the parent directory of "app" into sys.path
# so that Python recognizes "app" as an importable package.
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
parent_dir = os.path.abspath(os.path.join(notebook_dir, ".."))
sys.path.append(parent_dir)

In [ ]:
from dotenv import load_dotenv
load_dotenv()  # This looks for .env in the current working directory

In [3]:
import time

import pandas as pd
import seaborn as sns
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    answer_correctness,
)

In [4]:
# Benchmark data
test_df = pd.read_csv('benchmark.csv', delimiter=";")
test_df.head()

,question,ground_truth
0,What are some contracts that ended after 1990-...,There are a total of 377 contracts that ended ...
1,What are some contracts that ended after 1990-...,"There are 48 contracts classified as ""Strategi..."
2,"What are some contracts with HC2 Holdings, Inc.?","There is one contract involving HC2 Holdings, ..."
3,Which year was the most contracts signed?,The year with the most contracts signed was 20...
4,Which company has the most active contracts?,The company with the most active contracts is ...


In [5]:
from backend.src.agent import get_agent

In [6]:
agent = get_agent()

In [7]:
from langchain_core.messages import HumanMessage

input_messages = [HumanMessage(content="Which year was the most contracts signed?")]
messages = agent.invoke({"messages": input_messages})
for m in messages["messages"]:
    m.pretty_print()

================================ Human Message =================================

Which year was the most contracts signed?
================================== Ai Message ==================================
Tool Calls:
  ContractSearch (57b52df8-a533-42d4-9605-44f2b770fbed)
 Call ID: 57b52df8-a533-42d4-9605-44f2b770fbed
  Args:
    cypher_aggregation: RETURN effective_date.year AS year, count(*) AS count ORDER BY year DESC LIMIT 1
================================= Tool Message =================================
Name: ContractSearch

[{"year": 2024, "count": 1}]
================================== Ai Message ==================================

The year with the most contracts signed was 2024, with a total of 1 contract.


In [8]:
# History
messages

{'messages': [HumanMessage(content='Which year was the most contracts signed?', additional_kwargs={}, response_metadata={}, id='035402e1-d8c3-4870-882e-63645e8c18ec'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'ContractSearch', 'arguments': '{"cypher_aggregation": "RETURN effective_date.year AS year, count(*) AS count ORDER BY year DESC LIMIT 1"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-3d779455-a291-4e4e-a3ce-64c099d8545f-0', tool_calls=[{'name': 'ContractSearch', 'args': {'cypher_aggregation': 'RETURN effective_date.year AS year, count(*) AS count ORDER BY year DESC LIMIT 1'}, 'id': '57b52df8-a533-42d4-9605-44f2b770fbed', 'type': 'tool_call'}], usage_metadata={'input_tokens': 555, 'output_tokens': 26, 'total_tokens': 581, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content='[{"year": 2024, "count": 1}]', name='ContractSearch', id='7d83a0bf-a8a6-

In [9]:
# Chat with history

input_messages = messages['messages'] + [HumanMessage(content="Which party has the most active contracts?")]
messages = agent.invoke({"messages": input_messages})
for m in messages["messages"]:
    m.pretty_print()

================================ Human Message =================================

Which year was the most contracts signed?
================================== Ai Message ==================================
Tool Calls:
  ContractSearch (57b52df8-a533-42d4-9605-44f2b770fbed)
 Call ID: 57b52df8-a533-42d4-9605-44f2b770fbed
  Args:
    cypher_aggregation: RETURN effective_date.year AS year, count(*) AS count ORDER BY year DESC LIMIT 1
================================= Tool Message =================================
Name: ContractSearch

[{"year": 2024, "count": 1}]
================================== Ai Message ==================================

The year with the most contracts signed was 2024, with a total of 1 contract.
================================ Human Message =================================

Which party has the most active contracts?
================================== Ai Message ==================================
Tool Calls:
  ContractSearch (0b269a1b-afa8-4fd8-a606-3ab505f2bc86)
 

In [10]:
def get_answer(input: str) -> str:
    input_messages = [HumanMessage(content=input)]
    messages = agent.invoke({"messages": input_messages})
    return messages["messages"][-1].content

In [11]:
get_answer("What's your name?")

'I am Gemini, a large language model built by Google.'

In [12]:
from tqdm import tqdm

# Get workflows generated answers
results = []
latencies = []
for i, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Processing questions"):
    question = row['question']
    start = time.time()
    try:
        data = get_answer(question)
    except:
        data = {"answer": "timeout", "question": question}
    end = time.time()
    latencies.append(end - start)
    results.append(data)

Processing questions: 100%|██████████| 45/45 [01:08<00:00,  1.52s/it]


In [13]:
test_df["latencies"] = latencies
test_df["response"] = results


In [23]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)  # Adjust width to prevent wrapping
pd.set_option('display.max_colwidth', None)  # Prevents truncation of long column values

In [24]:
test_df[['response','question']]

,response,question
0,"I found 377 contracts that ended after 1990-01-01. Here are a few examples: a service contract (Nuveen - Remarketing Agreement) for remarketing shares which ends on 2024-03-31, a distributor contract (LUCIDINC_04_15_2011-EX-10.9-DISTRIBUTOR AGREEMENT) for Lucid products which ends on 2026-01-01, and a joint venture contract (MFAFINANCIAL,INC_07_06_2020-EX-99.D-JOINT FILING AGREEMENT) which ends on 2021-07-06.",What are some contracts that ended after 1990-01-01?
1,"Here are some strategic alliance contracts that ended after 1990-01-01. I am including a few examples of these contracts:\n* **ELFBEAUTY,INC\_07\_02\_2020-EX-10.1-COOPERATION AGREEMENT**: This is a Cooperation Agreement between e.l.f. Beauty, Inc. and Marathon Partners Equity Management, LLC. It includes details about board nomination, support period covenants, and standstill covenants. The contract was effective from 2020-7-1 to 2021-11-1.\n* **HC2HOLDINGS,INC\_05\_14\_2020-EX-10.1-COOPERATION AGREEMENT**: This is a Cooperation Agreement that modifies the composition of the Company's board of directors and other matters. It was effective from 2020-5-13 to 2021-1-1, with a monetary value of 352290.25.\n* **ROCKYMOUNTAINCHOCOLATEFACTORY,INC\_12\_23\_2019-EX-10.2-STRATEGIC ALLIANCE AGREEMENT**: A strategic cooperation alliance, effective from 2019-12-20 to 2021-12-20, by and among Farids & Co. LLC, Edible Arrangements, LLC, and Rocky Mountain Chocolate Factory, Inc.\n* **BERKELEYLIGHTS,INC\_06\_26\_2020-EX-10.12-COLLABORATION AGREEMENT**: A COLLABORATION AGREEMENT effective as of September 13th, 2019 is between Ginkgo Bioworks, Inc. and Berkeley Lights, Inc. to develop and deploy workflows on the Beacon Platform to accelerate the engineering of microbial organisms and mammalian cell lines and ends on 2026-9-30\n\nThere are a total of 48 contracts.",What are some contracts that ended after 1990-01-01 that fall under strategic alliance?
2,"I found one strategic alliance type contract with HC2 Holdings, Inc. The contract was effective from 2020-05-13 to 2021-01-01, with a monetary value of 352290.25. The contract scope is an agreement to modify the composition of the Company's board of directors and as to certain other matters, according to the summary.","What are some contracts with HC2 Holdings, Inc.?"
3,"The years 2018 and 2020 were the years in which the most contracts were signed. In both years, 50 contracts were signed.",Which year was the most contracts signed?
4,"HOF Village, LLC has the most active contracts with a total of 2.",Which company has the most active contracts?
5,"I looked for any contracts containing the word ""indemnification"" in the summary, and I didn't find any.",Do we have any contracts about indemnification?
6,"To provide the contracts with the highest total value, I would need to know how the monetary value is stored for each contract. Is it a field that I can access and sort by?\n\nIf you can provide more information about how the monetary value is stored or calculated, I can try to provide a more specific query.",Which Hosting contracts have the highest total value?
7,Microgenics Corporation has 1 contract.,How many contracts does Microgenics Corporation have?
8,I can search for contracts by country. Would you like me to search for contracts with the country set to 'US'?,What contracts are governed by US law?
9,"The party with the most Strategic Alliance contracts is VerticalNet, Inc. with 3 contracts.",Which party has the most Strategic Alliance contracts?


In [15]:
dataset = Dataset.from_pandas(test_df)

result = evaluate(
    dataset,
    metrics=[
        answer_correctness
    ],
)

Evaluating:   0%|          | 0/45 [00:00<?, ?it/s]

Exception raised in Job[13]: KeyError('reference')
Exception raised in Job[9]: KeyError('reference')
Exception raised in Job[7]: KeyError('reference')
Exception raised in Job[11]: KeyError('reference')
Exception raised in Job[8]: KeyError('reference')
Exception raised in Job[15]: KeyError('reference')
Exception raised in Job[6]: KeyError('reference')
Exception raised in Job[12]: KeyError('reference')
Exception raised in Job[16]: KeyError('reference')
Exception raised in Job[17]: KeyError('reference')
Exception raised in Job[14]: KeyError('reference')
Exception raised in Job[18]: KeyError('reference')
Exception raised in Job[20]: KeyError('reference')
Exception raised in Job[22]: KeyError('reference')
Exception raised in Job[25]: KeyError('reference')
Exception raised in Job[23]: KeyError('reference')
Exception raised in Job[26]: KeyError('reference')
Exception raised in Job[28]: KeyError('reference')
Exception raised in Job[30]: KeyError('reference')
Exception raised in Job[10]: KeyErr

In [16]:
result.to_pandas()

,user_input,response,reference,answer_correctness
0,What are some contracts that ended after 1990-...,I found 377 contracts that ended after 1990-01...,There are a total of 377 contracts that ended ...,0.523993
1,What are some contracts that ended after 1990-...,Here are some strategic alliance contracts tha...,"There are 48 contracts classified as ""Strategi...",0.373698
2,"What are some contracts with HC2 Holdings, Inc.?",I found one strategic alliance type contract w...,"There is one contract involving HC2 Holdings, ...",0.649150
3,Which year was the most contracts signed?,The years 2018 and 2020 were the years in whic...,The year with the most contracts signed was 20...,0.489641
4,Which company has the most active contracts?,"HOF Village, LLC has the most active contracts...",The company with the most active contracts is ...,0.994188
5,Do we have any contracts about indemnification?,I looked for any contracts containing the word...,There are currently no contracts in our system...,0.727548
6,Which Hosting contracts have the highest total...,To provide the contracts with the highest tota...,NaN,NaN
7,How many contracts does Microgenics Corporatio...,Microgenics Corporation has 1 contract.,NaN,NaN
8,What contracts are governed by US law?,I can search for contracts by country. Would y...,NaN,NaN
9,Which party has the most Strategic Alliance co...,The party with the most Strategic Alliance con...,NaN,NaN
